# Export workflow to Neurodata Without Borders file and upload to DANDI

In [10]:
import datajoint as dj
from tests.tutorial_pipeline import lab, subject, session, probe, ephys
from element_animal.export.nwb import subject_to_nwb
from element_lab.export.nwb import element_lab_to_nwb_dict
from element_session.export.nwb import session_to_nwb
from element_array_ephys.export.nwb import ecephys_session_to_nwb
from element_interface.dandi import upload_to_dandi

In [2]:
dj.conn()

DataJoint connection (connected) root@fakeservices.datajoint.io:3306

In [24]:
lab_key={"lab": "LabA"}
protocol_key={"protocol": "ProtA"}
project_key={"project": "ProjA"}
session_key = dict(subject="subject5", session_datetime="2023-01-01 00:00:00")

lab.Lab.insert1(
    {
        **lab_key,
        "lab_name": "LabA",
        "address": "",
        "time_zone": "UTC+0",
    },
    skip_duplicates=True,
)
lab.Organization.insert1(
    {
        "organization": "DataJoint",
        "org_name": "DataJoint",
        "org_address": "4265, San Felipe St., Houston, TX"
    }, skip_duplicates=True,
)
lab.Lab.Organization.insert1(
    {
        **lab_key,
        "organization": "DataJoint"
    }, skip_duplicates=True,
)
lab.ProtocolType.insert1({"protocol_type": "A"}, skip_duplicates=True)
lab.Protocol.insert1({**protocol_key, "protocol_type": "A"}, skip_duplicates=True)
lab.Project.insert1(project_key, skip_duplicates=True)

In [27]:
session.SessionNote.insert1(
    {
        **session_key,
        "session_note": "DataJoint ephys tutorial session."
    }
)

In [28]:
print('Lab:\n')
print(element_lab_to_nwb_dict(lab_key=lab_key, protocol_key=protocol_key, 
                              project_key=project_key))
print('\nAnimal:\n')
print(subject_to_nwb(session_key=session_key))
print('\nSession:\n')
print(session_to_nwb(session_key=session_key))

nwbfile = ecephys_session_to_nwb(session_key=session_key,
                                 raw=True,
                                 spikes=True,
                                 lfp="dj",
                                 end_frame=100,
                                 lab_key=lab_key,
                                 project_key=project_key,
                                 protocol_key=protocol_key,
                                 nwbfile_kwargs=None)

Lab:

{'institution': 'DataJoint', 'lab': 'LabA', 'experiment_description': '', 'keywords': None, 'related_publications': None, 'protocol': 'ProtA', 'notes': ''}

Animal:

subject pynwb.file.Subject at 0x140263087158656
Fields:
  age__reference: birth
  date_of_birth: 2023-01-01 00:00:00+00:00
  description: {"subject": "subject5", "subject_nickname": "", "sex": "U", "subject_birth_date": "2023-01-01", "subject_description": "", "species": null, "line": null, "strain": null, "source": null}
  sex: U
  species: []
  subject_id: subject5


Session:

root pynwb.file.NWBFile at 0x140263088511200
Fields:
  file_create_date: [datetime.datetime(2024, 4, 19, 15, 5, 15, 957866, tzinfo=tzlocal())]
  identifier: 83720c8f-64eb-4d6d-92c3-9d20deca359a
  session_description: DataJoint ephys tutorial session.
  session_id: subject5_2023-01-01T00:00:00
  session_start_time: 2023-01-01 00:00:00+00:00
  subject: subject pynwb.file.Subject at 0x140263087158656
Fields:
  age__reference: birth
  subject_id:

/usr/local/lib/python3.9/site-packages/pynwb/file.py:146: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['date_of_birth'] = _add_missing_timezone(date_of_birth)


TypeError: pynwb.file.NWBFile() argument after ** must be a mapping, not NoneType

In [12]:
(lab.Lab * lab.Lab.Organization * lab.Organization & lab_key)

lab Abbreviated lab name,organization Abbreviated organization name,lab_name Full lab name,address Physical lab address,"time_zone 'UTC±X' format or timezone, e.g., America/New_York",org_name Full organization name,org_address Address of the organization,org_comment Additional notes on the organization


In [ ]:
username_as_prefix = dj.config["database.user"] + "_"
if not dj.config['custom']:
    dj.config['custom'] = {}
dj.config['custom'].update({
        "ephys_mode": "no-curation",
        "database.prefix": username_as_prefix,
        })

If you haven't already populated the `lab`, `subject`, `session`, `probe`, and `ephys` schemas, please do so now with [04-automate](./04-automate-optional.ipynb). Note: exporting `ephys` data is currently only supported on the `no_curation` schema. 

In [ ]:
from workflow_array_ephys.pipeline import lab, subject, session, probe, ephys
from workflow_array_ephys.export import (element_lab_to_nwb_dict, subject_to_nwb, 
                                         session_to_nwb, ecephys_session_to_nwb, 
                                         write_nwb)
from element_interface.dandi import upload_to_dandi

## Export to NWB

We'll use the following keys to demonstrate export functions.

In [ ]:
lab_key={"lab": "LabA"}
protocol_key={"protocol": "ProtA"}
project_key={"project": "ProjA"}
session_key={"subject": "subject5",
             "session_datetime": "2018-07-03 20:32:28"}

### Upstream Elements

If you plan to use all upstream Elements, you can skip to the following section. To combine with other schemas, the following functions may be helpful.

- **Element Lab** `element_lab_to_nwb_dict` exports NWB-relevant items to `dict` format.
- **Element Animal** `subject_to_nwb` returns an NWB file with subject information.
- **Element Session** `session_to_nwb` returns an NWB file with subject and session information.

Note: `pynwb` will display a warning regarding timezone information - datetime fields are assumed to be in local time, and will be converted to UTC.


In [ ]:
## If you don't already have data in the Element Lab
lab.Lab.insert1(
    {
        **lab_key,
        "lab_name": "LabA",
        "institution": "",
        "address": "",
        "time_zone": "UTC+0",
    },
    skip_duplicates=True,
)
lab.ProtocolType.insert1({"protocol_type": "A"})
lab.Protocol.insert1({**protocol_key, "protocol_type": "A"}, skip_duplicates=True)
lab.Project.insert1(project_key, skip_duplicates=True)

In [9]:
from element_lab.export import 
print('Lab:\n')
print(element_lab_to_nwb_dict(lab_key=lab_key, protocol_key=protocol_key, 
                              project_key=project_key))
print('\nAnimal:\n')
print(subject_to_nwb(session_key=session_key))
print('\nSession:\n')
print(session_to_nwb(session_key=session_key))

nwbfile = ecephys_session_to_nwb(session_key=session_key,
                                 raw=True,
                                 spikes=True,
                                 lfp="dj",
                                 end_frame=100,
                                 lab_key=lab_key,
                                 project_key=project_key,
                                 protocol_key=protocol_key,
                                 nwbfile_kwargs=None)

TypeError: pynwb.file.NWBFile() argument after ** must be a mapping, not NoneType

In [8]:
protocol_key

{'protocol': 'ProtA'}

In [ ]:
print('Lab:\n')
print(element_lab_to_nwb_dict(lab_key=lab_key, protocol_key=protocol_key, 
                              project_key=project_key))
print('\nAnimal:\n')
print(subject_to_nwb(session_key=session_key))
print('\nSession:\n')
print(session_to_nwb(session_key=session_key))

### Element Array Electrophysiology

`ecephys_session_to_nwb` provides a full export mechanism, returning an NWB file with raw data, spikes, and LFP. Optional arguments determine which pieces are exported. For demonstration purposes, we recommend limiting `end_frame`.


In [ ]:
help(ecephys_session_to_nwb)

In [ ]:
nwbfile = ecephys_session_to_nwb(session_key=session_key,
                                 raw=True,
                                 spikes=True,
                                 lfp="dj",
                                 end_frame=100,
                                 lab_key=lab_key,
                                 project_key=project_key,
                                 protocol_key=protocol_key,
                                 nwbfile_kwargs=None)

In [ ]:
nwbfile

`write_nwb` can then be used to write this file to disk. The following cell will include a timestamp in the filename.

In [ ]:
import time
my_path = "./"
my_path = f"/home/{dj.config['database.user']}/" # for codebook users
write_nwb(nwbfile, my_path+time.strftime("_test_%Y%m%d-%H%M%S.nwb"))

## DANDI Upload

`element-interface.dandi` includes the `upload_to_dandi` utility to support direct uploads. For more information, see [DANDI documentation](https://www.dandiarchive.org/handbook/10_using_dandi/).

In order to upload, you'll need...
1. A DANDI account
2. A `DANDI_API_KEY`
3. A `dandiset_id`

These values can be added to your `dj.config` as follows:

In [ ]:
dj.config['custom']['dandiset_id']="<six digits as string>" 
dj.config['custom']['dandi.api']="<40-character alphanumeric string>"

This would facilitate routine updating of your dandiset.

In [ ]:
upload_to_dandi(
    data_directory="./temp_nwb/",
    dandiset_id=dj.config['custom']['dandiset_id'],
    staging=True,
    working_directory="./temp_nwb/",
    api_key=dj.config['custom']['dandi.api'],
    sync=False)